In [ ]:
# Insert code here.
!pip install cython
!pip install numpy
!pip install -r globalfn/requirements.txt
!pip install gspread
!pip install oauth2client

     |████████████████████████████████| 2.0 MB 3.3 MB/s eta 0:00:01
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 5.7 MB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 66.0 MB/s eta 0:00:01
     |████████████████████████████████| 136 kB 48.5 MB/s eta 0:00:01
     |████████████████████████████████| 981 kB 47.9 MB/s eta 0:00:01
     |████████████████████████████████| 24.2 MB 36.1 MB/s eta 0:00:01
     |████████████████████████████████| 788 kB 55.6 MB/s eta 0:00:01
     |████████████████████████████████| 660 kB 72.0 MB/s eta 0:00:01
     |████████████████████████████████| 68 kB 7.8 MB/s  eta 0:00:01
     |████████████████████████████████| 769 kB 51.8 MB/s eta 0:00:01
     |████████████████████████████████| 964 kB 52.1 MB/s eta 0:00:01
     |████████████████████████████████| 270 k

In [ ]:
from globalfn.full_text import full_text
import pathlib
import subprocess

# Google Sheets Use
import ast
import pandas as pd
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
# load documents of text (sentence by sentence)
for lang in ['de', 'el', 'en', 'fr', 'hi', 'jp', 'pt', 'se', 'ur']:
    path = pathlib.Path(f'vecalign_globalfn_data/{lang}/{lang}.txt')
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(f'vecalign_globalfn_data/{lang}/{lang}.txt', 'w+') as wf:
        for (_, sent) in full_text(lang):
            wf.write(sent + '\n')

In [ ]:
!./vecalign/overlap.py -i vecalign_globalfn_data/de/de.txt -o vecalign_globalfn_data/de/overlaps.de -n 10
!./vecalign/overlap.py -i vecalign_globalfn_data/el/el.txt -o vecalign_globalfn_data/el/overlaps.el -n 10
!./vecalign/overlap.py -i vecalign_globalfn_data/en/en.txt -o vecalign_globalfn_data/en/overlaps.en -n 10
!./vecalign/overlap.py -i vecalign_globalfn_data/fr/fr.txt -o vecalign_globalfn_data/fr/overlaps.fr -n 10
!./vecalign/overlap.py -i vecalign_globalfn_data/hi/hi.txt -o vecalign_globalfn_data/hi/overlaps.hi -n 10
!./vecalign/overlap.py -i vecalign_globalfn_data/jp/jp.txt -o vecalign_globalfn_data/jp/overlaps.jp -n 10
!./vecalign/overlap.py -i vecalign_globalfn_data/pt/pt.txt -o vecalign_globalfn_data/pt/overlaps.pt -n 10
!./vecalign/overlap.py -i vecalign_globalfn_data/se/se.txt -o vecalign_globalfn_data/se/overlaps.se -n 10
!./vecalign/overlap.py -i vecalign_globalfn_data/ur/ur.txt -o vecalign_globalfn_data/ur/overlaps.ur -n 10

In [ ]:
# create embeddings
!bash vecalign_globalfn.sh

You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
running install
running bdist_egg
running egg_info
writing fastBPE.egg-info/PKG-INFO
writing dependency_links to fastBPE.egg-info/dependency_links.txt
writing top-level names to fastBPE.egg-info/top_level.txt
package init file 'fastBPE/__init__.py' not found (or not a regular file)
reading manifest file 'fastBPE.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'fastBPE.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
running build_ext
creating build/bdist.linux-x86_64/egg
copying build/lib.linux-x86_64-3.7/fastBPE.cpython-37m-x86_64-linux-gnu.so -> build/bdist.linux-x86_64/egg
creating stub load

In [ ]:
# sentence alignment using `vecalign`
languages = ['de', 'el', 'en', 'fr', 'hi', 'jp', 'pt', 'se', 'ur']
for i in range(len(languages)):
    for j in range(i + 1, len(languages)):
        lang1 = languages[i]
        lang2 = languages[j]
        command = f"./vecalign/vecalign.py --alignment_max_size 8 --src vecalign_globalfn_data/{lang1}/{lang1}.txt --tgt vecalign_globalfn_data/{lang2}/{lang2}.txt --src_embed vecalign_globalfn_data/{lang1}/overlaps.{lang1} vecalign_globalfn_data/{lang1}/overlaps.{lang1}.emb --tgt_embed vecalign_globalfn_data/{lang2}/overlaps.{lang2} vecalign_globalfn_data/{lang2}/overlaps.{lang2}.emb > vecalign_globalfn_data/alignments/{lang1}-{lang2}.txt"
        subprocess.run(command, shell=True)

In [ ]:
def iter_pd(df):
    """
    Iterate through a dataframe
    """
    for val in df.columns:
        yield val
    for row in df.to_numpy():
        for val in row:
            if pd.isna(val):
                yield ""
            else:
                yield val

def pandas_to_sheets(pandas_df, sheet, clear = True):
    """
    Updates all values in a Google Sheets workbook to match a pandas dataframe
    """
    if clear:
        sheet.clear()
    (row, col) = pandas_df.shape
    cells = sheet.range("A1:{}".format(gspread.utils.rowcol_to_a1(row + 1, col)))
    for cell, val in zip(cells, iter_pd(pandas_df)):
        cell.value = val
    sheet.update_cells(cells)

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('vecalign_globalfn_data/gspread_service.json', scope)

gc = gspread.authorize(credentials)
workbook = gc.open_by_key("1W7tPyE2kiAziFOw-woaYzlDlC5aCc1jco-5VkAtEtCs")

In [ ]:
"""
Store the sentence alignment results on a Google Sheet.
"""
paths = pathlib.Path("/home/jovyan/work/vecalign_globalfn_data/alignments/").glob("*.txt")
for path in list(sorted(paths)):
    with open(path, "r+") as rf:
        alignments = dict()
        for line in rf:
            src, tgt, _ = line.split(':')
            src = ast.literal_eval(src)
            tgt = ast.literal_eval(tgt)
            alignments[tuple(src)] = tuple(tgt)
        
        # store the alignment results in Panda.DataFrame
        lang1, lang2 = path.stem.split("-")
        lang1_idx2id = {i:item for i, item in enumerate(full_text(lang1))}
        lang2_idx2id = {i:item for i, item in enumerate(full_text(lang2))}
        pd_data = {f"{lang1}_id":[], f"{lang1}_Sentence":[], f"{lang2}_id":[], f"{lang2}_Sentence":[]}
        for src, tgt in alignments.items():
            for src_idx in src:
                for tgt_idx in tgt:
                    pd_data[f"{lang1}_id"].append(lang1_idx2id[src_idx][0])
                    pd_data[f"{lang2}_id"].append(lang2_idx2id[tgt_idx][0])
                    pd_data[f"{lang1}_Sentence"].append(lang1_idx2id[src_idx][1])
                    pd_data[f"{lang2}_Sentence"].append(lang2_idx2id[tgt_idx][1])

        # store the DataFrame result in a Googel Sheet
        df = pd.DataFrame(data=pd_data)
        sheet = workbook.add_worksheet(title=f"{lang1.upper()}-{lang2.upper()}", rows="300", cols="10")
        pandas_to_sheets(df, sheet)

### Quantitative Analysis

In [ ]:
"""
Evaluate the accuracy of the automated sentence alignments.
"""
import collections
from globalfn.alignments import aligned_with

paths = pathlib.Path("/home/jovyan/work/vecalign_globalfn_data/alignments/").glob("*.txt")
for path in paths:
    for lang_alignment in ("de-en", "en-pt"):
        if lang_alignment in str(path):
            with open(path, "r+") as rf:
                # retrieve sentence alignments
                alignments = dict()
                for line in rf:
                    src, tgt, _ = line.split(':')
                    src = ast.literal_eval(src)
                    tgt = ast.literal_eval(tgt)
                    alignments[tuple(src)] = tuple(tgt)
                
                # store the alignment results into Panda Dataframe (for easier read)
                lang1, lang2 = path.stem.split("-")
                lang1_idx2id = {i:item for i, item in enumerate(full_text(lang1))}
                lang2_idx2id = {i:item for i, item in enumerate(full_text(lang2))}
                pd_data = {f"{lang1}_id":[], f"{lang1}_Sentence":[], f"{lang2}_id":[], f"{lang2}_Sentence":[]}
                for src, tgt in alignments.items():
                    for src_idx in src:
                        for tgt_idx in tgt:
                            pd_data[f"{lang1}_id"].append(lang1_idx2id[src_idx][0])
                            pd_data[f"{lang2}_id"].append(lang2_idx2id[tgt_idx][0])
                            pd_data[f"{lang1}_Sentence"].append(lang1_idx2id[src_idx][1])
                            pd_data[f"{lang2}_Sentence"].append(lang2_idx2id[tgt_idx][1])
                
                # store the alignment results into a dictionary for easier sentence ID access 
                D = collections.defaultdict(list)
                for i in range(len(pd_data[f"{lang1}_id"])):
                    D[pd_data[f"{lang1}_id"][i]].append(pd_data[f"{lang2}_id"][i])
                
                # evaluate all the alignments
                count = true = 0
                for src_id in D.keys():
                    _, res = aligned_with(src_id, lang2)
                    if res == D[src_id]:
                        true += 1
                    if res is not None:
                        count += 1
                
                print(f"Accuracy ({lang1}-{lang2}):", true/count)


Accuracy (de-en): 0.9772727272727273
Accuracy (en-pt): 0.9662921348314607
